In [ ]:
!pip install tensorflow --upgrade
!pip install openpyxl
!pip install PyArabic
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git &> /dev/null
!pip install emoji 
!pip install Arabic-Stopwords
!pip install tkseem
!pip install tnkeeh
!pip3 install fr-word-segment
!pip install pyspellchecker

## Importing libraries

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from keras.preprocessing.text import Tokenizer

import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from fastai.text.all import *

import sklearn
import regex as reg
from unicodedata import normalize
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch.optim.lr_scheduler import CyclicLR
from torchvision import models

from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,recall_score,f1_score
import os
import gensim


# keras
from keras.preprocessing.text import Tokenizer
from keras.layers import GRU,MaxPooling1D,GlobalMaxPooling1D,GlobalAveragePooling1D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras import callbacks
from keras.utils.vis_utils import plot_model

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 
from sklearn.metrics import roc_auc_score, accuracy_score,roc_curve, auc, plot_confusion_matrix, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from tensorflow.keras.models import Sequential
from sklearn.manifold import TSNE
from sklearn.naive_bayes import MultinomialNB
from keras.preprocessing.text import Tokenizer
import emoji

from keras.models import Model
from imblearn.over_sampling import SMOTE
import seaborn as sn
import pyarabic.araby as ar
import nltk
from nltk.tokenize import word_tokenize
import seaborn as sn
import tkseem as tk
import tnkeeh as tn
from nltk.stem.isri import ISRIStemmer
from spellchecker import SpellChecker
from wordsegment import load,segment
from keras.layers import Concatenate
import tensorflow as tf
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from fastai.text.all import *
load()

## Loading data

In [ ]:
df_ar = pd.read_csv('/kaggle/input/twitter/ar_dataset.csv')
df_fr = pd.read_csv('/kaggle/input/twitter/fr_dataset.csv')

## Analyzing data

In [ ]:
#### Class distribution 
cum = df_fr['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('my x label', size = 13)
ax.set_ylabel('my y label', size = 13)
ax.set_xticklabels("",size = 13)
sn.barplot(data=cum,x='HITId',y='target',palette = "rocket",ax=ax).set(title='Data distribution respect to class for French language') 
ax.set_xlabel('Category', size = 13)
plt.savefig('Data distribution for French language') 

#### Class distribution 
cum = df_ar['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel('my x label', size = 13)
ax.set_ylabel('my y label', size = 13)
ax.set_xticklabels("",size = 13)
sn.barplot(data=cum,x='HITId',y='target',palette = "rocket",ax=ax).set(title='Data distribution respect to class for Arabic language') 
ax.set_xlabel('Category', size = 13)
plt.savefig('Data distribution for Arabic language') 


In [ ]:
sn.set(font_scale=1)
print('French')
sn.catplot(x="directness", col="target", col_wrap=3,
                data=df_fr[df_fr.target.notnull()],
                kind="count", height=4.5,palette = "rocket", aspect=.8)

In [ ]:
print('Arabic')
sn.catplot(x="directness", col="target", col_wrap=3,
                data=df_ar[df_ar.target.notnull()],
                kind="count", height=4.5,palette = "rocket", aspect=.8)

In [ ]:
sn.set(font_scale=1)
print('French')
g = sn.catplot(x="group", col="target", col_wrap=3,
                data=df_fr[df_fr.target.notnull()],
                kind="count", height=4,palette = "rocket", aspect=.8)
g.set_xticklabels(rotation=60) 


In [ ]:
print('Arabic')
g= sn.catplot(x="group", col="target", col_wrap=3,
                data=df_ar[df_ar.target.notnull()],
                kind="count", height=4.5,palette = "rocket", aspect=.8)
g.set_xticklabels(rotation=60) 


## Preprocessing data

In [ ]:
def hash_fix(h):
    h1 = re.sub(r'[0-9]+', '', h)
    h2 = re.sub(r'#', '', h1)
    h3 = segment(str(h2))
    h4 = ' '.join(map(str, h3)) 
    return h4
tok_ar = tk.WordTokenizer()
tok_ar.train('/kaggle/input/twitter/ar_dataset.csv')
def prepro_ar(tweet):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    
    tweet = tweet.replace('user', '')
    tweet = tweet.replace('@user', '')
    tweet = re.sub( r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'',tweet)

    tweet = tweet.replace('url', '')
    tweet = re.sub(r'\u003c\\1', lambda m: "", tweet)
    tweet = re.sub(arabic_diacritics, '', str(tweet))
    tweet = re.sub(r'(.)\1+', "", tweet) 
    tweet = ar.strip_tashkeel(tweet)
    tweet = ar.strip_tatweel(tweet)
    tweet = tweet.replace("@", " ")
    tweet = tweet.replace("_", " ")
    tweet = re.sub("ى", "ي", tweet)
    tweet = re.sub("ؤ", "ء", tweet)
    tweet = re.sub("ئ", "ء", tweet)
    tweet = re.sub("ة", "ه", tweet)
    tweet = re.sub("گ", "ك", tweet)
    tweet = tweet.replace("آ", "ا")
    tweet = tweet.replace("إ", "ا")
    tweet = tweet.replace("أ", "ا")
    tweet = tweet.replace("ؤ", "و")
    tweet = tweet.replace("ئ", "ي")
    tweet = nltk.tokenize.word_tokenize(tweet)
    tweet = [ISRIStemmer().suf32(w) for w in tweet]
    for i in range(len(tweet)):
        if tweet[i] == 'اه':
            tweet[i] = 'الله'
    return tweet
contractions_fr = {
        'administration':'admin',
        'avec':'ac',
        'beaucoup':'bp',
        'c’est-à-dire':'cad',
        'cependant':'cpd',
        'chose':'ch',
        'conclusion':'ccl',
        'confer ':'cf',
        'court terme':'ct',
        'dans':'ds',
        'dedans':'dd',
        'définition':'déf',
        'et cetera':'etc',
        'être':'ê',
        'exemple':'ex',
        'extérieur':'ext',
        'font':'ft',
        'général':'gal',
        'gouvernement':'gouv',
        'grand':'gd',
        'groupe':'gp',
        'identique':'idel',
        'introduction':'intro',
        'jour':'jr',
        'long terme':'lt',
        'lorsque':'lsq',
        'mais':'ms',
        'même':'^m',
        'moyen terme':'mt',
        'nombre':'nb',
        'nombreux':'nbx',
        'nombre':'nb',
        'nombreux':'nbx',
        'observation':'obs',
        'ordre du jour':'oj',
        'page':'p',
        'parce que':'pcq',
        'pendant':'pdt',
        'personne':'pers',
        'point':'pt',
        'peut-être':'pê',
        'pour':'pr',
        'pourtant':'prtt',
        'quand':'qd',
        'quantité':'qté',
        'que':'q',
        'quelqu’un':'qqn',
        'quelque chose':'qqch',
        'quelque':'qq',
        'quelquefois':'qqf',
        'question':'quest',
        'rendez-vous':'rdv',
        'responsabilité':'respité',
        'seulement':'slt',
        'solution':'sol',
        'sont':'st',
        'sous':'ss',
        'souvent':'svt',
        'temps':'tps',
        'toujours':'tjrs',
        'tous':'ts',
        'tout':'tt',
        'toute':'tte',
        'toutes':'ttes',
        'vous':'vs',
        'le':'l\'',
        'me':'m\'',
        'de':'d\'',
        'te':'t\'',
        'se':'s\'',
        'ce':'c\'',
        'ne':'n\'',
        'que':'qu\'',
        'jusque':'jusqu\'',
        'lorsque':'lorsqu\'',
        'puisque':'puisqu\'',
        'quelque':'quelqu\'',
        'quoique':'quoiqu\'',
        'parce que':'parce qu\'',
        'tel que':'tel qu\'',
        'telle que':'telle qu\'',       
        'faculte':'fac',
        'bien':'bin',    
        'attend':'att',
        'je': 'j\'',
        'rire':'ptdr',
        'rire':'lol',
        'rire':'lmfao',
        'putin':'ptn',
}
french_stopwords = nltk.corpus.stopwords.words('french')
lemmatizer = FrenchLefffLemmatizer()
def prepro_fr(tweet):
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # normalize unicode characters
    tweet = normalize('NFD', tweet).encode('ascii','ignore')
    tweet = tweet.decode('UTF-8')
    #demojize
    tweet = emoji.demojize(tweet)
    if "#" in tweet:
        tweet = hash_fix(tweet)
    tweet = tweet.replace('user', '')
    tweet = tweet.replace('@user', '')
    tweet = tweet.replace("@", " ")
    tweet = tweet.replace('url', '')
    tweet = tweet.replace('rt', '')
    tweet = re.sub( r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'',tweet)
    # convert to lower case
    tweet = tweet.lower()
    tweet = tweet.replace('\'', '\' ') 
    # remove punctuation
    tweet = re.sub(r'\u003c\\1', lambda m: "-" if m.group(0) == "-" else "", tweet)
    # tokenization
    tweet = nltk.tokenize.word_tokenize(tweet)
    #contractions
    tweet = [list(contractions_fr.keys())[list(contractions_fr.values()).index(word)] if word in contractions_fr.values() else word for word in tweet]
    
    # stop words
    tweet = [w for w in tweet if w not in french_stopwords]
    sc = 'j[a-z]*'
    v = '[^aeyouisch]*'
    for w in tweet:
        x = re.findall(sc,w)
        xx = re.findall(v,w)
        if len(x)!=0:
            w = x[0].replace('j', 'je ')
            if len(x)>=3:
                w = w.replace(xx[0], '')
    # remove non-printable chars form each token
    tweet = [re_print.sub('', w) for w in tweet]
    # lemmatization
    tweet = [lemmatizer.lemmatize(w) for w in tweet]
    return tweet

In [ ]:
df_ar.tweet = df_ar.tweet.apply(lambda t: prepro_ar(t))
df_fr.tweet = df_fr.tweet.apply(lambda t: prepro_fr(t))

In [ ]:
#encoding labels
le = preprocessing.LabelEncoder()
df_fr.sentiment = le.fit_transform(df_fr.sentiment)
df_fr.directness = le.fit_transform(df_fr.directness)
df_fr.target = le.fit_transform(df_fr.target)
df_fr.annotator_sentiment = le.fit_transform(df_fr.annotator_sentiment)
df_fr.group = le.fit_transform(df_fr.group)
df_fr = df_fr.sample(frac = 1)

#encoding labels
le = preprocessing.LabelEncoder()
df_ar.sentiment = le.fit_transform(df_ar.sentiment)
df_ar.directness = le.fit_transform(df_ar.directness)
df_ar.target = le.fit_transform(df_ar.target)
df_ar.annotator_sentiment = le.fit_transform(df_ar.annotator_sentiment)
df_ar.group = le.fit_transform(df_ar.group)
df_ar = df_ar.sample(frac = 1)

In [ ]:
X_train_fr,X_test_fr, y_train_fr,y_test_fr = train_test_split(df_fr.tweet,
                                                              df_fr.target.values,
                                                              test_size=0.1)

X_train_ar,X_test_ar, y_train_ar,y_test_ar = train_test_split(df_ar.tweet,
                                                              df_ar.target.values,
                                                              test_size=0.1)

In [ ]:
train_set = X_train_fr.to_frame()
train_set.insert(0, "target", y_train_fr, True)
fig,ax= plt.subplots(figsize=(12,5))
ax.set_xlabel('my x label', size = 13)
ax.set_ylabel('my y label', size = 13)
ax.set_xticklabels("",size = 13)
sn.countplot(x="target", data=train_set, palette='PuRd_r',ax=ax)
fig.suptitle('Training set distribution before resampling (French dataset) ', fontsize=10)
plt.savefig('Training set distribution before resampling (French dataset)')

In [ ]:
train_set_ar = X_train_ar.to_frame()
train_set_ar.insert(0, "target", y_train_ar, True)
fig,ax= plt.subplots(figsize=(12,5))
ax.set_xlabel('my x label', size = 13)
ax.set_ylabel('my y label', size = 13)
ax.set_xticklabels("",size = 13)
sn.countplot(x="target", data=train_set_ar, palette='PuRd_r',ax=ax)
fig.suptitle('Training set distribution before resampling (Arabic dataset) ', fontsize=10)
plt.savefig('Training set distribution before resampling (Arabic dataset)')

In [ ]:
#resampling
def resample_ar(X, y):
    Xnew = X.copy()
    ynew = y.copy()
    setnew = Xnew
    setnew.insert(0, "target", ynew, True)

    lab0,lab1,lab2,lab3,lab4 = setnew[setnew.target==0],setnew[setnew.target==1],setnew[setnew.target==2],setnew[setnew.target==3],setnew[setnew.target==4]
    c0,c1,c2,c3,c4 = setnew.target.value_counts()
    av = int((c0+c1+c2+c3+c4)/5)  
    lab0_sampled = lab0.sample(av, replace=True) 
    lab1_sampled = lab1.sample(av, replace=True)
    lab2_sampled = lab2.sample(av, replace=True) 
    lab3_sampled = lab3.sample(av, replace=True) 
    lab4_sampled = lab4.sample(av, replace=True) 
    
    setnew_resampled = pd.concat([lab0_sampled,lab1_sampled,lab2_sampled,lab3_sampled,lab4_sampled], axis=0)
    setnew_resampled = setnew_resampled.sample(frac = 1)
    ynew = setnew_resampled['target']
    Xnew = setnew_resampled.drop(columns=['target'])
    return Xnew, ynew

def resample_fr(X, y):
    Xnew = X.copy()
    ynew = y.copy()
    setnew = Xnew
    setnew.insert(0, "target", ynew, True)
    lab0,lab1,lab2,lab3,lab4,lab5 = setnew[setnew.target==0],setnew[setnew.target==1],setnew[setnew.target==2],setnew[setnew.target==3],setnew[setnew.target==4],setnew[setnew.target==5]
    c0,c1,c2,c3,c4,c5 = setnew.target.value_counts()
    av = int((c0+c1+c2+c3+c4+c5)/6)  
    lab0_sampled = lab0.sample(av, replace=True) 
    lab1_sampled = lab1.sample(av, replace=True)
    lab2_sampled = lab2.sample(av, replace=True) 
    lab3_sampled = lab3.sample(av, replace=True) 
    lab4_sampled = lab4.sample(av, replace=True) 
    lab5_sampled = lab5.sample(av, replace=True) 
    setnew_resampled = pd.concat([lab0_sampled,lab1_sampled,lab2_sampled,lab3_sampled,lab4_sampled,lab5_sampled], axis=0)
    setnew_resampled = setnew_resampled.sample(frac = 1)
    ynew = setnew_resampled['target']
    Xnew = setnew_resampled.drop(columns=['target'])
    return Xnew, ynew


In [ ]:
X_train_fr_re,y_train_fr_re = resample_fr(X_train_fr.to_frame(),y_train_fr)
X_train_ar_re,y_train_ar_re = resample_ar(X_train_ar.to_frame(),y_train_ar)

In [ ]:
dic = {0:'disability', 
       1:'gender', 
       2:'origin', 
       3:'other', 
       4:'religion',
       5:'sexual_orientation',}

In [ ]:
train_set = X_train_fr_re.copy()
train_set.insert(0, "target", y_train_fr_re, True)
train_set.target = np.array([dic[t] for t in y_train_fr_re])
fig,ax= plt.subplots(figsize=(12,5))
ax.set_xlabel('', size = 13)
ax.set_ylabel('', size = 13)
ax.set_xticklabels("",size = 13)
sn.countplot(x="target", data=train_set, palette='PuRd_r',ax=ax)
fig.suptitle('Training set distribution after resampling (French dataset) ', fontsize=15)
plt.savefig('Training set distribution after resampling (French dataset)')

In [ ]:
train_set_ar = X_train_ar_re.copy()
train_set_ar.insert(0, "target", y_train_ar_re, True)
train_set_ar.target = np.array([dic[t] for t in y_train_ar_re])
fig,ax= plt.subplots(figsize=(12,5))
ax.set_xlabel('', size = 13)
ax.set_ylabel('', size = 13)
ax.set_xticklabels("",size = 13)
sn.countplot(x="target", data=train_set_ar, palette='PuRd_r',ax=ax)
fig.suptitle('Training set distribution after resampling (Arabic dataset) ', fontsize=15)
plt.savefig('Training set distribution after resampling (Arabic dataset)')

In [ ]:
class_weights_train_fr = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train_fr),
                                        y = y_train_fr                                                    
                                    )
class_weights_train_fr = dict(zip(np.unique(y_train_fr), class_weights_train_fr))

print('Class weights for french dataset: '+'\n',class_weights_train_fr)

In [ ]:
class_weights_train_ar = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train_ar),
                                        y = y_train_ar                                                    
                                    )
class_weights_train_ar = dict(zip(np.unique(y_train_ar), class_weights_train_ar))

print('Class weights for Arabic dataset: '+'\n',class_weights_train_ar)

In [ ]:
y_train_fr, y_test_fr = pd.get_dummies(y_train_fr), pd.get_dummies(y_test_fr)
y_train_ar, y_test_ar = pd.get_dummies(y_train_ar), pd.get_dummies(y_test_ar)

y_train_fr_re = pd.get_dummies(y_train_fr_re)
y_train_ar_re = pd.get_dummies(y_train_ar_re)

## Embedding

In [ ]:
emd_vect =  300
def embedding_mat(file, word_index):
    embedding_dict = {}
    for line in file:
        values=line.split()
        word=values[0]
        if word in word_index.keys():
            vector = np.asarray(values[1:], 'float32')
            embedding_dict[word] = vector

    num_words=len(word_index)+1
    embedding_matrix=np.zeros((num_words, emd_vect))
    for word,i in tqdm(word_index.items()):
        vect=embedding_dict.get(word)
        if vect is not None:
            embedding_matrix[i]  = vect[:emd_vect]
    return embedding_matrix

In [ ]:
f_fr = open('/kaggle/input/vectorization/cc.fr.300.vec', encoding='utf-8')
f_ar = open('/kaggle/input/vectorization-ar/cc.ar.300.vec', encoding='utf-8')

In [ ]:
X_train_fr = list(X_train_fr)
X_test_fr = list(X_test_fr)

X_train_ar = list(X_train_ar)
X_test_ar = list(X_test_ar)


tok_fr = tf.keras.preprocessing.text.Tokenizer()
tok_fr.fit_on_texts(df_fr.tweet.values)
#text to integer sequence
X_train_fr = tok_fr.texts_to_sequences(X_train_fr)
X_test_fr = tok_fr.texts_to_sequences(X_test_fr)

tok_ar = tf.keras.preprocessing.text.Tokenizer()
tok_ar.fit_on_texts(df_ar.tweet.values)
word_index_ar = tok_ar.word_index

#text to integer sequence
X_train_ar = tok_ar.texts_to_sequences(X_train_ar)
X_test_ar = tok_ar.texts_to_sequences(X_test_ar)


#counting the length of the tweet and taking the max
tweets_length_fr= [len(X_train_fr[i]) for i in range(len(X_train_fr))]
max_seq_len_fr = max(tweets_length_fr)
print("French max sequence length",max_seq_len_fr)
#counting the length of the tweet and taking the max
tweets_length_ar= [len(X_train_ar[i]) for i in range(len(X_train_ar))]
max_seq_len_ar = max(tweets_length_ar)
print("Arabic max sequence length",max_seq_len_ar)


#padding
X_train_fr1 = pad_sequences(X_train_fr, maxlen=max_seq_len_fr)
X_test_fr1 = pad_sequences(X_test_fr, maxlen=max_seq_len_fr)
#X_train_fr2 = X_train_fr[['sentiment', 'directness', 'group']].values
#X_test_fr2 = X_test_fr[['sentiment', 'directness', 'group']].values
embedding_matrix_fr = embedding_mat(f_fr, tok_fr.word_index)
embedding_matrix_fr

#padding
X_train_ar1 = pad_sequences(X_train_ar, maxlen=max_seq_len_ar)
X_test_ar1 = pad_sequences(X_test_ar, maxlen=max_seq_len_ar)
#X_train_ar2 = X_train_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
#X_test_ar2 = X_test_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
embedding_matrix_ar = embedding_mat(f_ar, tok_ar.word_index)
embedding_matrix_ar

In [ ]:
f_fr = open('/kaggle/input/vectorization/cc.fr.300.vec', encoding='utf-8')
f_ar = open('/kaggle/input/vectorization-ar/cc.ar.300.vec', encoding='utf-8')

In [ ]:
X_train_fr_re = list(X_train_fr_re.tweet.values)
X_train_ar_re = list(X_train_ar_re.tweet.values)

tok_fr = tf.keras.preprocessing.text.Tokenizer()
tok_fr.fit_on_texts(df_fr.tweet.values)
#text to integer sequence
X_train_fr_re = tok_fr.texts_to_sequences(X_train_fr_re)

#text to integer sequence
X_train_ar_re = tok_ar.texts_to_sequences(X_train_ar_re)


#counting the length of the tweet and taking the max
tweets_length_fr_re= [len(X_train_fr_re[i]) for i in range(len(X_train_fr_re))]
max_seq_len_fr_re = max(tweets_length_fr_re)
print("French max sequence length",max_seq_len_fr_re)

tok_ar = tf.keras.preprocessing.text.Tokenizer()
tok_ar.fit_on_texts(df_ar.tweet.values)
#counting the length of the tweet and taking the max
tweets_length_ar_re= [len(X_train_ar_re[i]) for i in range(len(X_train_ar_re))]
max_seq_len_ar_re = max(tweets_length_ar_re)
print("Arabic max sequence length",max_seq_len_ar_re)


#padding
X_train_fr1_re = pad_sequences(X_train_fr_re, maxlen=max_seq_len_fr_re)
#X_train_fr2_re = X_train_fr_re[['sentiment', 'directness', 'group']].values
embedding_matrix_fr_re = embedding_mat(f_fr, tok_fr.word_index)
embedding_matrix_fr_re

#padding
X_train_ar1_re = pad_sequences(X_train_ar_re, maxlen=max_seq_len_ar_re)
#X_train_ar2 = X_train_ar_re[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
embedding_matrix_ar_re = embedding_mat(f_ar, tok_ar.word_index)
embedding_matrix_ar_re

## Classification models

In [ ]:
input_fr_1 = Input(shape=(max_seq_len_fr,))
nb_words_fr = len(tok_fr.word_index)+1
nb_classes_fr = 6

input_ar_1 = Input(shape=(max_seq_len_ar,))
nb_words_ar = len(tok_ar.word_index)+1
nb_classes_ar = 5

In [ ]:
class MODEL:
    def __init__(self,embedding_matrix,nb_classes,nb_words,out_dim = 300):
            self.embedding_matrix = embedding_matrix
            self.nb_classes = nb_classes
            self.nb_words = nb_words
            self.out_dim = out_dim
    
    def model(self):
        model = tf.keras.Sequential()
        embedding_layer = tf.keras.layers.Embedding(
                                self.nb_words, 
                                self.out_dim,  
                                weights=[self.embedding_matrix],
                                trainable=False,
                            )
        model.add(embedding_layer)
        model.add(tf.keras.layers.Bidirectional(GRU(128)))
        model.add(Dense(100,activation= 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(self.nb_classes, activation='softmax'))

        return model
        
    def train(self,model,num_epochs,batch_size,X,y):
        early = callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0, 
                            patience=3,
                            verbose=1, 
                            mode='auto')

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        
        history = model.fit(X,
                      y,
                      batch_size=batch_size,
                      epochs=num_epochs,
                      validation_split=0.25,
                      callbacks = [early],
                      verbose=1)
        return history
    
    
class MODEL_WE:
    def __init__(self,class_weights,embedding_matrix,nb_classes,nb_words,out_dim = 300):
            self.class_weights = class_weights
            self.embedding_matrix = embedding_matrix
            self.nb_classes = nb_classes
            self.nb_words = nb_words
            self.out_dim = out_dim
    
    def model(self):
        model = tf.keras.Sequential()
        embedding_layer = tf.keras.layers.Embedding(
                                self.nb_words, 
                                self.out_dim,  
                                weights=[self.embedding_matrix],
                                trainable=False,
                            )
        model.add(embedding_layer)
        model.add(tf.keras.layers.Bidirectional(LSTM(300)))
        model.add(Dense(600,activation= 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(150, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(self.nb_classes, activation='softmax'))

        return model
        
    def train(self,model,num_epochs,batch_size,X,y):
        early = callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=0, 
                            patience=3,
                            verbose=1, 
                            mode='auto')

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        
        history = model.fit(X,
                      y,
                      batch_size=batch_size,
                      epochs=num_epochs,
                      validation_split=0.25,
                      callbacks = [early],
                      class_weight=self.class_weights,
                      verbose=1)
        return history

In [ ]:
def plot_acc_loss(history):
    """
    Plot accuracy and loss of a model
    @params:
            - history: history of the model
    @return:
            plots
    """
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    l = list(history.history.keys())
    # accuracy plot
    ax[0].plot(history.history[l[1]])
    ax[0].plot(history.history[l[3]])
    ax[0].set_title('model accuracy')
    ax[0].set_ylabel('accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].legend(['train', 'val'], loc='upper left')
    # loss plot
    ax[1].plot(history.history[l[0]])
    ax[1].plot(history.history[l[2]])
    ax[1].set_title('model loss')
    ax[1].set_ylabel('loss')
    ax[1].set_xlabel('epoch')
    ax[1].legend(['train', 'val'], loc='upper left')
    fig.suptitle('Model results for resampled dataset')
    plt.savefig("Model results for resampled dataset")

In [ ]:
def eval_fr(model):
    pred = model.predict(X_test_fr1)
    pred_fr = [np.argmax(i) for i in pred]
    gt = y_test_fr.idxmax(axis=1).values
    # precision tp / (tp + fp)
    precision_fr = precision_score(list(gt), pred_fr,average='macro')
    # accuracy: (tp + tn) / (p + n)
    accuracy_fr = accuracy_score(list(gt), pred_fr)
    # recall: tp / (tp + fn)
    recall_fr = recall_score(list(gt), pred_fr, labels=[0,1,2,3,4,5],average='macro')
    # f1: 2 tp / (2 tp + fp + fn)
    f1_fr = f1_score(list(gt), pred_fr,labels=[0,1,2,3,4,5],average='weighted')
    return precision_fr,accuracy_fr,recall_fr,f1_fr

def eval_ar(model):
    pred_ar = model.predict(X_test_ar1)
    pred_ar = [np.argmax(i) for i in pred_ar]
    gt_ar = y_test_ar.idxmax(axis=1).values
    # precision tp / (tp + fp)
    precision_ar = precision_score(list(gt_ar), pred_ar,average='macro')
    # accuracy: (tp + tn) / (p + n)
    accuracy_ar = accuracy_score(list(gt_ar), pred_ar)
    # recall: tp / (tp + fn)
    recall_ar = recall_score(list(gt_ar), pred_ar, labels=[0,1,2,3,4],average='macro')
    # f1: 2 tp / (2 tp + fp + fn)
    f1_ar = f1_score(list(gt_ar), pred_ar,labels=[0,1,2,3,4],average='weighted')
    return precision_ar,accuracy_ar,recall_ar,f1_ar


In [ ]:
# MODEL WITHOUT WEIGHTS
mod = MODEL(embedding_matrix_fr,nb_classes_fr,nb_words_fr)
model = mod.model()
his = mod.train(model,30,32,X_train_fr1_re,y_train_fr_re)

In [ ]:
plot_acc_loss(his)

In [ ]:
pr_fr,acc_fr,rec_fr,f1_fr = eval_fr(model)

In [ ]:
mod_ar = MODEL(embedding_matrix_ar,nb_classes_ar,nb_words_ar)
model_ar = mod_ar.model()
his_ar = mod_ar.train(model_ar,30,32,X_train_ar1_re,y_train_ar_re)

In [ ]:
plot_acc_loss(his_ar)

In [ ]:
pr_ar,acc_ar,rec_ar,f1_ar = eval_ar(model_ar)

In [ ]:
pr_ar,acc_ar,rec_ar,f1_ar

In [ ]:
# MODEL WITH WEIGHTS
mod_we = MODEL_WE(class_weights_train_fr,embedding_matrix_fr,nb_classes_fr,nb_words_fr)
model_we = mod_we.model()
his_we = mod_we.train(model,30,32,X_train_fr1_re,y_train_fr_re)

In [ ]:
plot_acc_loss(his_we)

In [ ]:
pr_fr_we,acc_fr_we,rec_fr_we,f1_fr_we = eval_fr(model_we)

In [ ]:
pr_fr_we,acc_fr_we,rec_fr_we,f1_fr_we

In [ ]:
mod_ar_we = MODEL_WE(class_weights_train_ar,embedding_matrix_ar,nb_classes_ar,nb_words_ar)
model_ar_we = mod_ar_we.model()
his_ar_we = mod_ar_we.train(model_ar_we,30,32,X_train_ar1_re,y_train_ar_re)

In [ ]:
plot_acc_loss(his_ar_we)

In [ ]:
pr_ar_we,acc_ar_we,rec_ar_we,f1_ar_we = eval_ar(model_ar_we)

## Testing on new data

In [ ]:
def predicting(tweet,model):
    x = pd.DataFrame({"tweet":[tweet]})
    p = [prepro_fr(t) for t in x.tweet]
    x = tok_fr.texts_to_sequences(p)
    x = pad_sequences(x, maxlen=max_seq_len_fr)
    pred = np.argmax(model.predict(x))
    if pred == 0:
        return 'disability'
    elif pred==1:
        return 'gender'
    elif pred == 2:
        return 'origin'
    elif pred==3:
        return 'other'
    elif pred == 4:
        'religion'
    else:
        return'sexual_orientation'